In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pybdshadow
from shapely.geometry import Polygon

In [2]:
buildings = gpd.read_file(r'../example/data/Suzhoubuildings/苏州(1).shp')
buildings = pybdshadow.bd_preprocess(buildings)

In [3]:
buildings = gpd.read_file(r'../example/data/bd_demo.json')
buildings = pybdshadow.bd_preprocess(buildings)

In [3]:
from pybdshadow import *
from suncalc import get_position
date = pd.to_datetime('2015-01-01 02:45:33.959797119')
height = 'height'
ground = 0
epsg = 3857

def bdshadow_sunlight_vector(buildings, date,merge = False, height='height', ground=0, epsg=3857):

    building = buildings.copy()
    building[height] -= ground
    building = building[building[height] > 0]
    # calculate position
    lon1, lat1, lon2, lat2 = list(building.bounds.mean())
    lon = (lon1+lon2)/2
    lat = (lat1+lat2)/2
    # transform coordinate system
    building.crs = 'epsg:4326'
    building = building.to_crs(epsg=epsg)

    # obtain sun position
    sunPosition = get_position(date, lon, lat)
    buildingshadow = building.copy()
    # walls
    a = buildingshadow['geometry'].apply(lambda r:list(r.exterior.coords))
    buildingshadow['wall'] = a
    buildingshadow = buildingshadow.set_index(['building_id'])
    a = buildingshadow.apply(lambda x:pd.Series(x['wall']),axis = 1).unstack()
    walls = a[- a.isnull()].reset_index().sort_values(by = ['building_id','level_0'])
    walls = pd.merge(walls,buildingshadow['height'].reset_index())
    walls['x1'] = walls[0].apply(lambda r:r[0])
    walls['y1'] = walls[0].apply(lambda r:r[1])
    walls['x2'] = walls['x1'].shift(-1)
    walls['y2'] = walls['y1'].shift(-1)
    walls = walls[walls['building_id'] == walls['building_id'].shift(-1)]
    walls = walls[['x1','y1','x2','y2','building_id','height']]
    walls['wall'] = walls.apply(lambda r: [[r['x1'], r['y1']],
                                        [r['x2'], r['y2']]], axis=1)
    walls_shape = np.array(list(walls['wall']))
    # calculate shadow for walls
    shadowShape = calSunShadow_vector(walls_shape,walls['height'].values,sunPosition)
    walls['geometry'] = list(shadowShape)
    walls['geometry'] = walls['geometry'].apply(lambda r:Polygon(r))
    walls = gpd.GeoDataFrame(walls)

    walls.crs = 'epsg:'+str(epsg)
    shadows = walls[['building_id','geometry']].to_crs(epsg=4326)
    if merge:
        shadows = merge_shadow(shadows)
    return shadows

In [4]:
shadows = bdshadow_sunlight_vector(buildings, date)

12.9657142162323
37.269726037979126
50.474833726882935


In [5]:
pybdshadow.show_bdshadow(buildings = buildings,shadows = shadows)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
